In [3]:
import requests
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
headers = {"Accept-Language": "en-US, en;q=0.5"}
url = "https://www.imdb.com/search/title/?groups=top_1000&ref_=adv_prv"
results = requests.get(url, headers=headers)
soup = BeautifulSoup(results.text, "html.parser")

ConnectionError: HTTPSConnectionPool(host='www.imdb.com', port=443): Max retries exceeded with url: /search/title/?groups=top_1000&ref_=adv_prv (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x0000017C88085B48>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [2]:
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

In [3]:
movie_div = soup.find_all('div', class_='lister-item mode-advanced')

In [4]:
movie_div

[<div class="lister-item mode-advanced">
 <div class="lister-top-right">
 <div class="ribbonize" data-caller="filmosearch" data-tconst="tt6751668"></div>
 </div>
 <div class="lister-item-image float-left">
 <a href="/title/tt6751668/"> <img alt="Parasite" class="loadlate" data-tconst="tt6751668" height="98" loadlate="https://m.media-amazon.com/images/M/MV5BYWZjMjk3ZTItODQ2ZC00NTY5LWE0ZDYtZTI3MjcwN2Q5NTVkXkEyXkFqcGdeQXVyODk4OTc3MTY@._V1_UX67_CR0,0,67,98_AL_.jpg" src="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/large/film-184890147._CB466725069_.png" width="67"/>
 </a> </div>
 <div class="lister-item-content">
 <h3 class="lister-item-header">
 <span class="lister-item-index unbold text-primary">1.</span>
 <a href="/title/tt6751668/">Parasite</a>
 <span class="lister-item-year text-muted unbold">(2019)</span>
 </h3>
 <p class="text-muted">
 <span class="certificate">R</span>
 <span class="ghost">|</span>
 <span class="runtime">132 min</span>
 <span class="ghost">|</span>


In [5]:
for container in movie_div:
    name=container.h3.a.text
    titles.append(name)
    date=container.h3.find('span',class_='lister-item-year text-muted unbold').text
    years.append(date)
    runtime=container.p.find('span',class_='runtime').text if container.p.find('span',class_='runtime')else " "
    time.append(runtime)
    imdb=float(container.strong.text)
    imdb_ratings.append(imdb)
    m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
    metascores.append(m_score)
    nv=container.find_all('span',attrs={'name':'nv'})
    vote=nv[0].text
    votes.append(vote)
    gross=nv[1].text if len(nv)>1 else '-'
    us_gross.append(gross)
movies = pd.DataFrame({
'movie': titles,
'year': years,
'timeMin': time,
'imdb': imdb_ratings,
'metascore': metascores,
'votes': votes,
'us_grossMillions': us_gross,
})
print(movies)



                                                movie        year  timeMin  \
0                                            Parasite      (2019)  132 min   
1                                          Knives Out      (2019)  131 min   
2                                         Jojo Rabbit      (2019)  108 min   
3                                                1917      (2019)  119 min   
4                                       The Gentlemen      (2019)  113 min   
5                                          Uncut Gems      (2019)  135 min   
6                    Once Upon a Time... in Hollywood      (2019)  161 min   
7                                               Joker      (2019)  122 min   
8                                      Ford v Ferrari      (2019)  152 min   
9                                        Little Women      (2019)  135 min   
10                                     The Lighthouse  (I) (2019)  109 min   
11                                       The Irishman      (2019

In [6]:
movies['timeMin']=movies['timeMin'].str.extract('(\d+)').astype(int)

In [7]:
movies['metascore']=movies['metascore'].astype(int)

In [8]:
movies['votes']=movies['votes'].str.replace(',','').astype(int)

In [9]:
movies['us_grossMillions'] = movies['us_grossMillions'].map(lambda x: x.lstrip('$').rstrip('M'))
movies['us_grossMillions'] = pd.to_numeric(movies['us_grossMillions'], errors='coerce')

In [10]:
print(movies)

                                                movie        year  timeMin  \
0                                            Parasite      (2019)      132   
1                                          Knives Out      (2019)      131   
2                                         Jojo Rabbit      (2019)      108   
3                                                1917      (2019)      119   
4                                       The Gentlemen      (2019)      113   
5                                          Uncut Gems      (2019)      135   
6                    Once Upon a Time... in Hollywood      (2019)      161   
7                                               Joker      (2019)      122   
8                                      Ford v Ferrari      (2019)      152   
9                                        Little Women      (2019)      135   
10                                     The Lighthouse  (I) (2019)      109   
11                                       The Irishman      (2019

In [11]:
print(movies.dtypes)

movie                object
year                 object
timeMin               int32
imdb                float64
metascore             int32
votes                 int32
us_grossMillions    float64
dtype: object


In [12]:
movies['year']=movies['year'].str.replace('(','')
movies['year']=movies['year'].str.replace(')','')
print(movies['year'])

0       2019
1       2019
2       2019
3       2019
4       2019
5       2019
6       2019
7       2019
8       2019
9       2019
10    I 2019
11      2019
12      2019
13      2019
14      2019
15      1994
16      2017
17      1997
18      1972
19      2008
20      2016
21      2006
22      2009
23      2010
24      1994
25      2014
26      2017
27      1980
28      2018
29      2013
30      2001
31      2001
32      1990
33      2017
34      1999
35      2014
36      1982
37      2019
38      1999
39      1993
40      1991
41      2000
42      2019
43    I 2017
44      2014
45      2018
46      2018
47      1993
48      1994
49      1985
Name: year, dtype: object


In [13]:
movies.to_csv('movies.csv')